In [1]:
import requests
import json
import time
import random

# --- CONFIGURACIÓN DE LA INFRAESTRUCTURA ---
ORION_HOST = "localhost" # Reemplaza con la IP/nombre real de tu Orion Broker
ORION_PORT = "1026"
BASE_URL = f"http://{ORION_HOST}:{ORION_PORT}/v2/entities/"

# --- CONFIGURACIÓN DE ENTIDADES Y ATRIBUTOS ---
# Define los atributos de las 3 entidades y sus rangos de simulación
SENSOR_CONFIG = {
    "SensorCO201": {
        "attributes": {
            "CO2": {"min": 350, "max": 1000, "decimales": 0} # ppm
        }
    },
    "SensorCalidad1": {
        "attributes": {
            "ph": {"min": 6.5, "max": 8.5, "decimales": 1},
            "temperature": {"min": 18, "max": 28, "decimales": 1},
            "cloro": {"min": 0.5, "max": 5.0, "decimales": 1}
        }
    },
    "SensorTemp1": {
        "attributes": {
            "temperature": {"min": 20, "max": 35, "decimales": 1},
            "humedad": {"min": 40, "max": 90, "decimales": 0}
        }
    }
}

# --- FUNCIÓN PARA ENVIAR DATOS ---
def update_attribute(entity_id, attribute_name, value):
    """Envía una petición PATCH a Orion para actualizar un atributo específico."""
    
    entity_url = BASE_URL + entity_id + "/attrs"
    
    # Construir el payload JSON para la actualización (PATCH)
    payload = {
        attribute_name: {
            "value": value,
            "type": "Number" 
            # Asumimos que todos los valores a actualizar son de tipo Number
        }
    }
    
    headers = {"Content-Type": "application/json"}

    print(f"-> Enviando {entity_id} - {attribute_name}: {value}")
    
    # Enviar la petición PATCH
    try:
        response = requests.patch(entity_url, data=json.dumps(payload), headers=headers)
        
        if response.status_code == 204:
            # print(f"<- Éxito: Código {response.status_code}")
            pass
        else:
            print(f"<- !!! Error: Código {response.status_code}. Respuesta: {response.text}")
            
    except requests.exceptions.ConnectionError as e:
        print(f"!!! Error de Conexión: No se pudo conectar a Orion en {BASE_URL}. Verifica que el broker esté activo.")
        # Salir del bucle si hay un error de conexión
        raise e


# --- BUCLE DE CARGA MASIVA ---
def massive_data_load(updates_per_attribute=400, delay_sec=0.1):
    """Simula la toma de muestras de forma masiva para todas las entidades."""
    
    total_updates = updates_per_attribute * sum(len(config["attributes"]) for config in SENSOR_CONFIG.values())
    print(f"\n--- INICIANDO CARGA MASIVA ---")
    print(f"Objetivo: {updates_per_attribute} actualizaciones por atributo.")
    print(f"Total de peticiones a enviar: {total_updates}")
    
    # Bucle principal para las 400 iteraciones
    for i in range(updates_per_attribute):
        print(f"\n--- ITERACIÓN GLOBAL {i+1}/{updates_per_attribute} ---")
        
        # Bucle secundario para recorrer todas las entidades y sus atributos
        for entity_id, config in SENSOR_CONFIG.items():
            for attr_name, attr_config in config["attributes"].items():
                
                # 1. Generar un valor aleatorio dentro del rango definido
                random_value = round(
                    random.uniform(attr_config['min'], attr_config['max']), 
                    attr_config['decimales']
                )
                
                # 2. Enviar la actualización (PATCH)
                update_attribute(entity_id, attr_name, random_value)
                
                # 3. Esperar un breve intervalo para no saturar el sistema
                time.sleep(delay_sec)
    
    print("\n--- CARGA MASIVA FINALIZADA ---")

if __name__ == "__main__":
    massive_data_load()


--- INICIANDO CARGA MASIVA ---
Objetivo: 400 actualizaciones por atributo.
Total de peticiones a enviar: 2400

--- ITERACIÓN GLOBAL 1/400 ---
-> Enviando SensorCO201 - CO2: 718.0
-> Enviando SensorCalidad1 - ph: 7.8
-> Enviando SensorCalidad1 - temperature: 19.0
-> Enviando SensorCalidad1 - cloro: 1.9
-> Enviando SensorTemp1 - temperature: 24.4
-> Enviando SensorTemp1 - humedad: 67.0

--- ITERACIÓN GLOBAL 2/400 ---
-> Enviando SensorCO201 - CO2: 797.0
-> Enviando SensorCalidad1 - ph: 7.4
-> Enviando SensorCalidad1 - temperature: 22.6
-> Enviando SensorCalidad1 - cloro: 4.0
-> Enviando SensorTemp1 - temperature: 24.1
-> Enviando SensorTemp1 - humedad: 56.0

--- ITERACIÓN GLOBAL 3/400 ---
-> Enviando SensorCO201 - CO2: 673.0
-> Enviando SensorCalidad1 - ph: 7.2
-> Enviando SensorCalidad1 - temperature: 27.7
-> Enviando SensorCalidad1 - cloro: 3.0
-> Enviando SensorTemp1 - temperature: 25.4
-> Enviando SensorTemp1 - humedad: 55.0

--- ITERACIÓN GLOBAL 4/400 ---
-> Enviando SensorCO201 - 